# Backtesting using VectorBT

source: [Vectorbt GitHub](https://github.com/polakowo/vectorbt).

Vectorbt is a powerful Python library designed for efficient backtesting and analysis of quantitative trading strategies. It stands out for several reasons:
- **Vectorized Operations:** Leveraging NumPy and Pandas, Vectorbt utilizes vectorized operations for faster and more efficient computations, enhancing the speed of backtesting.
- **Flexible Data Management:** It offers a flexible and user-friendly interface for handling financial time series data, allowing users to easily manipulate and preprocess data for analysis.
- **Modular Components:** Vectorbt is built with a modular structure, enabling users to customize and extend functionalities based on their specific needs. This modularity promotes code reusability and scalability.
- **Performance Metrics:** The library provides a comprehensive set of performance metrics, enabling traders to assess the profitability and risk of their strategies thoroughly.
- **Interactive Visualizations:** With built-in plotting capabilities using Plotly and Matplotlib, Vectorbt allows for interactive and customizable visualizations, aiding in the interpretation of backtesting results.
- **Strategy Composition:** Users can easily compose complex trading strategies by combining various signals, indicators, and rules in a concise and expressive manner.
- **Risk Management:** Vectorbt includes tools for implementing robust risk management strategies, essential for controlling drawdowns and maximizing long-term returns.
- **Event-Driven Backtesting:** It supports event-driven backtesting, allowing users to model and test strategies based on specific market events, providing a more realistic simulation.




### Basic Usage

In [203]:
import vectorbt as vbt 
import datetime 
import numpy as np

# end date 
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=7)

# set parameters for RSI
rsi_level_lower = 30
rsi_level_upper = 70


# download data for microsoft
ticker_price = vbt.YFData.download('MSFT', start=start_date, end=end_date,missing_index='drop',interval='1m').get('Close')


# create RSI
rsi = vbt.RSI.run(ticker_price, 14)

# create entries and exits 
entries  = rsi.rsi_crossed_below(rsi_level_lower)
exits = rsi.rsi_crossed_above(rsi_level_upper)

# create portfolio
pf = vbt.Portfolio.from_signals(ticker_price, entries, exits, freq='1m')

# print stats
print(pf.stats())

# show portfolio
pf.plot().show()


Start                         2024-02-05 14:30:00+00:00
End                           2024-02-09 20:59:00+00:00
Period                                  1 days 08:27:00
Start Value                                       100.0
End Value                                    101.208375
Total Return [%]                               1.208375
Benchmark Return [%]                           2.679592
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.870657
Max Drawdown Duration                   0 days 15:51:00
Total Trades                                         22
Total Closed Trades                                  22
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  68.181818
Best Trade [%]                                 0.525865
Worst Trade [%]                                -

### Custom indicators

In [204]:
# define a custom indicator
def custom_idc(close,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30):

    # close is the input data in numpy array and has number of columns equal to number of assets

    # rsi calculation
    rsi = vbt.RSI.run(close,window=rsi_window).rsi.to_numpy()

    # moving average
    ma = vbt.MA.run(close,window=ma_window).ma.to_numpy()

    # create signal sell=-1,hold=0, buy=1

    # sell if rsi>rsi_level_upper
    trend = np.where(rsi>rsi_level_upper,-1,0)

    # buy if rsi<rsi_level_lower and close>ma
    trend = np.where((rsi<rsi_level_lower)&(close<ma) ,1,trend)

    return trend

# setup indicator
ind = (vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window','rsi_level_upper','rsi_level_lower'],
    output_names=['value'])
    .from_apply_func(custom_idc,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30))


res = ind.run(ticker_price,rsi_window=14,ma_window=60,rsi_level_upper=rsi_level_upper,rsi_level_lower=rsi_level_lower)

entries = res.value == 1
exits = res.value == -1

# create portfolio
pf = vbt.Portfolio.from_signals(ticker_price, entries, exits, freq='1m')

# print stats
print(pf.stats())

# show portfolio
pf.plot().show()



Start                         2024-02-05 14:30:00+00:00
End                           2024-02-09 20:59:00+00:00
Period                                  1 days 08:27:00
Start Value                                       100.0
End Value                                    100.917867
Total Return [%]                               0.917867
Benchmark Return [%]                           2.679592
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.431881
Max Drawdown Duration                   0 days 11:20:00
Total Trades                                         19
Total Closed Trades                                  19
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  68.421053
Best Trade [%]                                 0.311799
Worst Trade [%]                               -0

Of course as is evident this simple strategy is not very succesfull. It has major drawdown periods and less returns.

Let's instead of taking closing value try to take an average value over a week for getting rsi.


### Custom Indicator

In [223]:
# define a custom indicator
def custom_idc(close,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30):

   
    # use last element of the resampled data
    close_resampled = close.resample("5T").last()
    
    # # Update the index of the resampled data
    # close_resampled.index = adjusted_resampled_timestamps
    # rsi calculation
    rsi = vbt.RSI.run(close_resampled, window=rsi_window).rsi
    rsi.index = close_resampled.index
    rsi = rsi[(rsi_window,'Close')]
    rsi.name = 'Close'
    rsi = rsi.to_frame()
   
    rsi_aligned,_ = rsi.align(close,
                            broadcast_axis=0,
                            method='ffill',
                            join='right')
    
    close = close.to_numpy()
    rsi_aligned = rsi_aligned.to_numpy()
    
    # moving average
    ma = vbt.MA.run(close,window=ma_window).ma.to_numpy()

    # # create signal sell=-1,hold=0, buy=1
    # # sell if rsi>rsi_level_upper
    trend = np.where(rsi_aligned>rsi_level_upper,-1,0)
    # buy if rsi<rsi_level_lower and close>ma
    trend = np.where((rsi_aligned<rsi_level_lower)&(close<ma) ,1,trend)
    #trend=1
    return trend

# setup indicator
ind = (vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window','rsi_level_upper','rsi_level_lower'],
    output_names=['value'])
    .from_apply_func(custom_idc,
        rsi_window=14,
        ma_window=50,
        rsi_level_upper=70,
        rsi_level_lower=30,
        # keep pandas 
        keep_pd=True))


res = ind.run(ticker_price,rsi_window=5,ma_window=60,rsi_level_upper=rsi_level_upper,rsi_level_lower=rsi_level_lower)

entries = res.value == 1
exits = res.value == -1

# create portfolio
pf = vbt.Portfolio.from_signals(ticker_price, entries, exits, freq='1m')

# print stats
print(pf.stats())

# show portfolio
pf.plot().show()



Start                         2024-02-05 14:30:00+00:00
End                           2024-02-09 20:59:00+00:00
Period                                  1 days 08:27:00
Start Value                                       100.0
End Value                                    101.179206
Total Return [%]                               1.179206
Benchmark Return [%]                           2.679592
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.694381
Max Drawdown Duration                   0 days 12:38:00
Total Trades                                         19
Total Closed Trades                                  19
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  73.684211
Best Trade [%]                                 0.782399
Worst Trade [%]                               -0

# Hyperparameter Tuning

In [6]:
# define a custom indicator
def custom_idc(close,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30):

   
    # use last element of the resampled data
    close_resampled = close.resample("5T").last()
    
    # # Update the index of the resampled data
    # close_resampled.index = adjusted_resampled_timestamps
    # rsi calculation
    rsi = vbt.RSI.run(close_resampled, window=rsi_window).rsi
    rsi.index = close_resampled.index
    print(rsi)
    rsi = rsi[(rsi_window,'Close')]
    # rsi.name = 'Close'
    # rsi = rsi.to_frame()
   
    # rsi_aligned,_ = rsi.align(close,
    #                         broadcast_axis=0,
    #                         method='ffill',
    #                         join='right')
    
    # close = close.to_numpy()
    # rsi_aligned = rsi_aligned.to_numpy()
    
    # # moving average
    # ma = vbt.MA.run(close,window=ma_window).ma.to_numpy()

    # # # create signal sell=-1,hold=0, buy=1
    # # # sell if rsi>rsi_level_upper
    # trend = np.where(rsi_aligned>rsi_level_upper,-1,0)
    # # buy if rsi<rsi_level_lower and close>ma
    # trend = np.where((rsi_aligned<rsi_level_lower)&(close<ma) ,1,trend)
    # #trend=1
    # return trend


import vectorbt as vbt 
import datetime 
import numpy as np

# end date 
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=7)

# set parameters for RSI
rsi_level_lower = 30
rsi_level_upper = 70


# download data for microsoft
ticker_price = vbt.YFData.download('MSFT', start=start_date, end=end_date,missing_index='drop',interval='1m').get('Close')




# setup indicator
ind = (vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window','rsi_level_upper','rsi_level_lower'],
    output_names=['value'])
    .from_apply_func(custom_idc,
        rsi_window=14,
        ma_window=50,
        rsi_level_upper=70,
        rsi_level_lower=30,
        # keep pandas 
        keep_pd=True))



# set parameters for RSI
rsi_level_lower = 30
rsi_level_upper = 70

res = ind.run(ticker_price,
              rsi_window=[5,10,12,14],
              ma_window=[50,60,70,90],
              rsi_level_upper=rsi_level_upper,
              rsi_level_lower=rsi_level_lower)

entries = res.value == 1
exits = res.value == -1

# create portfolio
pf = vbt.Portfolio.from_signals(ticker_price, entries, exits, freq='1m')

# print stats
print(pf.total_return())



rsi_window                         5
                               Close
Datetime                            
2024-02-05 14:30:00+00:00        NaN
2024-02-05 14:35:00+00:00        NaN
2024-02-05 14:40:00+00:00        NaN
2024-02-05 14:45:00+00:00        NaN
2024-02-05 14:50:00+00:00        NaN
...                              ...
2024-02-09 20:35:00+00:00  98.075117
2024-02-09 20:40:00+00:00  97.672438
2024-02-09 20:45:00+00:00  42.859812
2024-02-09 20:50:00+00:00  40.425946
2024-02-09 20:55:00+00:00  67.250857

[1230 rows x 1 columns]
rsi_window                        10
                               Close
Datetime                            
2024-02-05 14:30:00+00:00        NaN
2024-02-05 14:35:00+00:00        NaN
2024-02-05 14:40:00+00:00        NaN
2024-02-05 14:45:00+00:00        NaN
2024-02-05 14:50:00+00:00        NaN
...                              ...
2024-02-09 20:35:00+00:00  64.150400
2024-02-09 20:40:00+00:00  58.919204
2024-02-09 20:45:00+00:00  52.524926
2024-02-09 20

AssertionError: (1, 4) and (1947, 4) do not match